In [2]:
from ppsim import Simulation, StatePlotter, time_trials
%matplotlib widget
%load_ext line_profiler
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as widgets
from tqdm import tqdm, trange
import time

In [3]:
from typing import NamedTuple
from math import log2, ceil

class Agent(NamedTuple):
    value: int
    x: int
    y: int
    output: int
        
def initial_agent(opinion, n):
    L = 2 ** ceil(log2(n))
    if opinion.lower() == 'a':
        return Agent(value = L, x = L, y = 0, output = 1)
    elif opinion.lower() == 'b':
        return Agent(value = -L, x = 0, y = L, output = -1)
    else:
        raise ValueError('opinion must be "a" or "b"')
        
def reduce(u, v):
    if u == v:
        return 0, 0
    elif u == 2 * v:
        return u - v, 0
    elif 2 * u == v:
        return 0, v - u
    else:
        return u, v
    
def cancel(x1, y1, x2, y2):
    x1o, y2o = reduce(x1, y2)
    x2o, y1o = reduce(x2, y1)
    return x1o, y1o, x2o, y2o

def join(x1, y1, x2, y2):
    if x1 - y1 > 0 and x2 - y2 > 0 and y1 == y2:
        y1o, y2o = y1 + y2, 0
    else:
        y1o, y2o = y1, y2
    if x1 - y1 < 0 and x2 - y2 < 0 and x1 == x2:
        x1o, x2o = x1 + x2, 0
    else:
        x1o, x2o = x1, x2
    return x1o, y1o, x2o, y2o

def split(x1, y1, x2, y2):
    if (x1 - y1 > 0 or x2 - y2 > 0) and max(x1, x2) > 1 and min(x1, x2) == 0:
        x1o = x2o = max(x1, x2) // 2
    else:
        x1o, x2o = x1, x2
    if (x1 - y1 < 0 or x2 - y2 < 0) and max(y1, y2) > 1 and min(y1, y2) == 0:
        y1o = y2o = max(y1, y2) // 2
    else:
        y1o, y2o = y1, y2
    return x1o, y1o, x2o, y2o
        
def normalize(x, y, v):
    x1, y1 = reduce(x, y)
    if x1 == y1 == 0:
        if v >= 0:
            output = +1
        else:
            output = -1
    else:
        if x1 > y1:
            output = +1
        else:
            output = -1
    return Agent(x = x1, y = y1, output = output, value = x1 - y1)
    
def split_join_majority(a, b):
    if a.x == a.y == b.x == b.y == 0:
        return a, b
    else:
        x1, y1, x2, y2 = split(*join(*cancel(a.x, a.y, b.x, b.y)))
    return normalize(x1, y1, x2 - y2), normalize(x2, y2, x1 - y1)

In [6]:
# implement test that checks for convergence more rarely, and then zooms in one

def output_dominant_convergence_check(config):
    outputs = {state.output for state in config.keys()}
    return len(outputs) == 1

def initial_conditions(n):
    return {initial_agent('a',n): n // 2 + 1, initial_agent('b', n): n // 2 - 1}

In [4]:
def state_map(state):
    if state.x == 0 and state.y != 0:
        return 'pure y'
    elif state.y == 0 and state.x != 0:
        return 'pure x'
    elif state.x == state.y == 0:
        return 'zero'
    else:
        return 'mixed'

In [9]:
n = 10 ** 6
sim = Simulation(initial_conditions(n), split_join_majority)

In [10]:
sp = StatePlotter(state_map = state_map)
sim.add_snapshot(sp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
sim.run()
sim.snapshot_slider()

interactive(children=(IntSlider(value=0, description='index', layout=Layout(width='100%'), max=98), Output()),…

In [17]:
display(sp.fig.canvas)
sp.ax.set_yscale('symlog')
sim.snapshot_slider()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='index', layout=Layout(width='100%'), max=88), Output()),…

In [16]:
sim.reset()
sim.run()

In [5]:
df = time_trials(split_join_majority, [int(n) for n in np.geomspace(10 ** 1,  10 ** 6, 20)], initial_conditions, output_dominant_convergence_check)

  0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
plt.figure()
sns.lineplot(x="n", y="time", data=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='n', ylabel='time'>

In [8]:
d = {'n':[], 'stabilization time':[]}
num_trials = 100
ns = [int(n) for n in np.geomspace(10 ** 1,  10 ** 10, 50)]
RUN_TIME = 60 * 60 * 7
end = time.perf_counter() + RUN_TIME
for i in trange(len(ns)):
    n = ns[i]
    sim = Simulation({initial_agent('a',n): n // 2 + 1, initial_agent('b', n): n // 2}, split_join_majority)
    t = time.perf_counter()
    time_limit = t + (end - t) / (len(ns) - i)
    j = 0
    while j < num_trials and time.perf_counter() < time_limit:
        j += 1
        sim.reset({initial_agent('a',n): n // 2 + 1, initial_agent('b', n): n // 2})
        sim.run(converged, interval = 0.01)
        d['n'].append(n)
        d['stabilization time'].append(sim.time)

df = pd.DataFrame(data=d)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

  2%|█▋                                                                                 | 1/50 [00:00<00:18,  2.72it/s]

  4%|███▎                                                                               | 2/50 [00:00<00:16,  2.90it/s]

  6%|████▉                                                                              | 3/50 [00:01<00:17,  2.67it/s]

 Time: 7.9436

...

 86%|█████████████████████████████████████████████████████████████████▎          | 43/50 [5:23:42<2:21:58, 1216.94s/it]

 88%|██████████████████████████████████████████████████████████████████▉         | 44/50 [5:47:33<2:08:05, 1280.97s/it]

 90%|████████████████████████████████████████████████████████████████████▍       | 45/50 [6:21:24<2:05:30, 1506.06s/it]

 92%|█████████████████████████████████████████████████████████████████████▉      | 46/50 [7:03:09<2:00:23, 1805.77s/it]

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [7:04:49<00:00, 509.78s/it]


In [9]:
df

,n,stabilization time
0,10,6.454545
1,10,5.818182
2,10,4.181818
3,10,6.181818
4,10,4.454545
...,...,...
3137,339322177,118.382534
3138,517947467,120.293401
3139,790604321,128.868080
3140,1206792640,134.516064


In [10]:
plt.figure()
sns.scatterplot(x="n", y="stabilization time", data=df)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.figure()
sns.lineplot(x="n", y="stabilization time", data=df)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.figure()
sns.regplot(x="n", y="stabilization time", data=df,
                 x_estimator=np.mean, logx=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='n', ylabel='stabilization time'>

In [260]:
from natsort import natsorted
sim.history
['bias', 'role']

# map
def state_map(state):
    if state.role == 'Clock':
        if state.minute < 20:
            return 'early clock'
        else:
            return 'late clock'
    else:
        if state.bias == -1:
            return 'opinion B'
        if state.bias == 1:
            return 'opinion A'
        if state.bias == 0:
            return 'unbiased'
        
def state_map(state):
    if state.x == 0 and state.y != 0:
        return 'pure y'
    elif state.y == 0 and state.x != 0:
        return 'pure x'
    elif state.x == state.y == 0:
        return 'zero'
    else:
        return 'mixed'

categories = natsorted(set([state_map(state) for state in sim.state_list if state_map(state) is not None]))
categories_dict = {j: i for i, j in enumerate(categories)}
matrix = np.zeros((len(categories), len(sim.state_list)), dtype=np.int64)
for i, state in enumerate(sim.state_list):
    m = state_map(state)
    if m is not None:
        matrix[categories_dict[m], i] += 1

plt.figure()
sns.barplot(x = categories, y = np.matmul(sim.config_array, matrix.T))
matrix.shape

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(4, 422)

In [261]:
pd.DataFrame(np.matmul(sim.history.to_numpy(), matrix.T), columns = categories, index = sim.history.index).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='time'>

In [256]:
sim.run(100.)

KeyboardInterrupt: 

In [205]:
from ppsim import Snapshot
class BiasPlotter(Snapshot):

    def initialize(self):
        self.fig, self.ax = plt.subplots()

    def update(self, index=None):
        super().update(index)
        self.ax.clear()
        self.simulation.history.T.groupby(level=('bias','role')).sum().T.plot(ax = self.ax)
        if index is not None:
            self.ax.axvline(x=self.time)
        self.fig.canvas.draw()

In [226]:
pd.concat([sim.history.xs(-1, level='bias', axis = 1, drop_level=False), sim.history.xs(1, level='bias', axis = 1, drop_level=False)])

role      Main                                               ...            \
minute     NaN                                               ...             
hour       NaN                                               ...             
exponent   -34  -33  -32  -31  -30  -29  -28  -27  -26  -25  ...  -9   -8    
bias        -1   -1   -1   -1   -1   -1   -1   -1   -1   -1  ...    1    1   
time                                                         ...             
0.000000     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
1.000002     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
2.000055     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
3.000166     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
4.000334     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
5.000746     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
6.001314     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
7.001899     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
8.002428     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
9.003060     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
10.003687    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
11.004303    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
12.005088    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
13.005922    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
14.006591    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
15.007272    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
16.007972    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
17.008649    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
18.009434    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
19.010201    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
20.010926    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
21.011788    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
22.012601    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
0.000000   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
1.000002   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
2.000055   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
3.000166   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
4.000334   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
5.000746   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
6.001314   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
7.001899   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
8.002428   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
9.003060   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
10.003687  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
11.004303  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
12.005088  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
13.005922  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
14.006591  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
15.007272  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
16.007972  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
17.008649  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
18.009434  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
19.010201  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
20.010926  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
21.011788  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
22.012601  NaN  NaN  N

In [27]:
h.iloc[1].groupby(level = ['x', 'output']).sum()

x     output
0     -1        382
       1        160
1     -1          0
       1          0
2     -1          0
       1          0
4     -1          0
       1          0
8     -1          2
       1          0
16    -1          2
       1          3
32    -1          6
       1          6
64    -1         12
       1          7
128   -1         26
       1         35
256   -1          0
       1         73
512    1        143
1024   1        144
Name: 1.0999000999000998, dtype: int64

In [8]:
h = sim.history

value,-1024,-1022,-1020,-1016,-1008,-992,-960,-896,-768,-512,...,512,768,896,960,992,1008,1016,1020,1022,1024
x,0,2,4,8,16,32,64,128,256,0,...,512,1024,1024,1024,1024,1024,1024,1024,1024,1024
y,1024,1024,1024,1024,1024,1024,1024,1024,1024,512,...,0,256,128,64,32,16,8,4,2,0
output,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,1
time,,,,,,,,,,,,,,,,,,,,,
0.000000,500,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,501
1.099900,133,0,0,0,0,0,0,2,0,127,...,99,0,1,0,0,0,0,0,0,143
2.329670,29,0,0,0,1,0,0,0,0,77,...,38,0,4,2,0,0,0,0,0,41
3.687313,12,0,0,0,0,1,0,0,0,28,...,24,0,3,1,1,1,0,0,0,7
5.045954,5,0,0,0,0,0,1,0,0,5,...,12,0,0,1,0,1,0,0,0,1
6.388611,1,0,0,0,0,1,0,0,0,4,...,4,0,0,0,0,0,0,0,0,0


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

d = {'n':[], 'silence time':[]}
num_trials = 1
for n in [int(n) for n in np.geomspace(10 ** 2,  10 ** 4, 20)]:
    print(f'n = {n}')
    sim = Simulation({initial_agent('a',n): n // 2 + 1, initial_agent('b', n): n // 2}, split_join_majority)
    print(sim)
    print(sim.sample_silence_time())
#     for _ in range(num_trials):
#         d['n'].append(n)
#         d['silence time'].append(sim.sample_silence_time())

# df = pd.DataFrame(data=d)

# sns.lineplot(x="n", y="silence time", data=df)

n = 100


NameError: name 'Simulation' is not defined

In [51]:
split_join_majority(a, c)

(Agent(x=16, y=0, output=1, value=16), Agent(x=16, y=0, output=1, value=16))

In [56]:
sim = Simulation({a: 100, b: 100}, split_join_majority)
sim.state_list

[Agent(x=0, y=0, output=-1, value=0),
 Agent(x=0, y=0, output=1, value=0),
 Agent(x=0, y=1, output=-1, value=-1),
 Agent(x=0, y=2, output=-1, value=-2),
 Agent(x=0, y=4, output=-1, value=-4),
 Agent(x=0, y=8, output=-1, value=-8),
 Agent(x=0, y=16, output=-1, value=-16),
 Agent(x=0, y=32, output=-1, value=-32),
 Agent(x=1, y=0, output=1, value=1),
 Agent(x=1, y=4, output=-1, value=-3),
 Agent(x=1, y=8, output=-1, value=-7),
 Agent(x=1, y=16, output=-1, value=-15),
 Agent(x=2, y=0, output=1, value=2),
 Agent(x=2, y=8, output=-1, value=-6),
 Agent(x=2, y=16, output=-1, value=-14),
 Agent(x=2, y=32, output=-1, value=-30),
 Agent(x=4, y=0, output=1, value=4),
 Agent(x=4, y=1, output=1, value=3),
 Agent(x=4, y=16, output=-1, value=-12),
 Agent(x=4, y=32, output=-1, value=-28),
 Agent(x=8, y=0, output=1, value=8),
 Agent(x=8, y=1, output=1, value=7),
 Agent(x=8, y=2, output=1, value=6),
 Agent(x=8, y=32, output=-1, value=-24),
 Agent(x=16, y=0, output=1, value=16),
 Agent(x=16, y=1, output=1

In [55]:
sorted([state.value for state in sim.state_list])

[-32,
 -30,
 -28,
 -24,
 -16,
 -15,
 -14,
 -12,
 -8,
 -7,
 -6,
 -4,
 -3,
 -2,
 -1,
 0,
 0,
 1,
 2,
 3,
 4,
 6,
 7,
 8,
 12,
 14,
 15,
 16,
 24,
 28,
 30,
 32]